In [1]:
import pandas as pd
import numpy as np

## Household Information

In [3]:
hh = pd.read_parquet('../../hces-2023-24/data/sec-4-HCQ.parquet')
hh.head()

,hh_size,employed_annual,nco_2015,nic_2008,max_income_from,hasincome_selfemp_agri,hasincome_wage_agri,hasincome_casual_agri,hh_type,hoh_religion,...,land_type,land_owned_in_acres,has_house,house_type,energy_source_cooking,energy_source_lighting,rationcard_type,is_rent,has_benefited_from_pmgky,multiplier
hhid,,,,,,,,,,,,,,,,,,,,,
22300101,4,yes,112.0,47522.0,self-employment,NaN,NaN,NaN,1,Hinduism,...,Homestead only,0.02,yes,owned,LPG,electricity (incl. generated by solar or wind ...,Above Poverty Line (APL),NaN,no,57436
22300201,4,yes,112.0,73100.0,self-employment,NaN,NaN,NaN,1,Hinduism,...,Homestead only,0.01,yes,hired,LPG,electricity (incl. generated by solar or wind ...,Above Poverty Line (APL),NaN,no,27497
22300202,2,no,NaN,NaN,None,NaN,NaN,NaN,9,Hinduism,...,Homestead only,0.01,yes,owned,LPG,electricity (incl. generated by solar or wind ...,Above Poverty Line (APL),NaN,no,27497
22300203,2,no,NaN,NaN,None,NaN,NaN,NaN,9,Hinduism,...,Homestead and other land,0.03,yes,owned,LPG,electricity (incl. generated by solar or wind ...,Above Poverty Line (APL),NaN,no,27497
22300204,2,yes,231.0,85301.0,regular wage/salary earning,NaN,NaN,NaN,2,Christianity,...,Homestead only,0.02,yes,owned,LPG,electricity (incl. generated by solar or wind ...,Above Poverty Line (APL),NaN,no,27497


In [4]:
# demographics + meals
sec3 = pd.read_parquet('../../hces-2023-24/data/sec-3-HCQ.parquet')
sec3.head()

rel_to_hoh  gender  age     marital_status  \
hhid     person_id                                                    
22300101 3          unmarried child    male   23      never married   
         2                   spouse  female   45  currently married   
         4          unmarried child  female   24      never married   
         1                     self    male   56  currently married   
22300201 2                   spouse  female   44  currently married   

                                  edu_level  edu_years used_internet  \
hhid     person_id                                                     
22300101 3                         graduate       18.0           yes   
         2                        secondary       10.0           yes   
         4          post graduate and above       20.0           yes   
         1                         graduate       15.0           yes   
22300201 2             upper primary/middle        8.0            no   

                    days_outside_home  meals_daily  meals_school  \
hhid     person_id                                                 
22300101 3                        0.0          3.0           NaN   
         2                        0.0          3.0           NaN   
         4                        0.0          3.0           NaN   
         1                        0.0          3.0           NaN   
22300201 2                        0.0          3.0           NaN   

                    meals_employer  meals_others  meals_payment  meals_home  \
hhid     person_id                                                            
22300101 3                     NaN           NaN            3.0        87.0   
         2                     NaN           3.0            NaN        87.0   
         4                     NaN           NaN            4.0        86.0   
         1                     NaN           NaN            3.0        87.0   
22300201 2                     NaN           NaN            NaN        90.0   

                    member_status  original_member  multiplier  
hhid     person_id                                              
22300101 3                      1              1.0       57436  
         2                      1              1.0       57436  
         4                      1              1.0       57436  
         1                      1              1.0       57436  
22300201 2                      1              1.0       27497

In [69]:
gb = sec3.groupby(sec3.index.get_level_values(0))

genders = gb['gender'].value_counts().unstack().fillna(value=0)
n_total = gb['gender'].count()
n_total.name = "n_total"
n_child = gb['age'].apply(lambda x: (x <= 15).sum())
n_child.name = "n_child"
n_elderly = gb['age'].apply(lambda x: (x >= 60).sum())
n_elderly.name = "n_elderly"

marital_status = gb['marital_status'].value_counts().unstack().fillna(value=0)
edu_level = gb['edu_level'].value_counts().unstack().fillna(value=0)
n_used_internet = gb['used_internet'].apply(lambda x: (x == 'yes').sum())
n_days_outside_home = gb['days_outside_home'].sum()

n_meals = gb[[c for c in sec3 if c.startswith('meals_')]].sum()
mult = gb['multiplier'].first()
mult.name = "mult"

In [70]:
hh_details = pd.concat([
    genders,
    n_total, n_child, n_elderly,
    marital_status, edu_level,
    n_used_internet, n_days_outside_home,
    n_meals, mult
], axis=1)

In [71]:
households = pd.concat([hh, hh_details], axis=1)
households.to_parquet('data/hh_details.parquet')

---

## Procurement

In [90]:
# ration cards, procurements against them
sec41 = pd.read_parquet('../../hces-2023-24/data/sec-4.1-FDQ.parquet')
sec41.head()

,used_ration_card,procured_rice,procured_wheat,procured_grain,procured_sugar,procured_pulses,procured_edible_oil,procured_other,online_groceries,online_milkprod,online_veg,online_fruits,online_dryfruits,online_eggfishmeat,online_processed_served,online_processed_packed,online_other,performed_ceremony,meals_served_to_nonhh,multiplier
hhid,,,,,,,,,,,,,,,,,,,,
22300101,no,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,no,5,57436
22300201,no,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,no,3,27497
22300202,no,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,no,2,27497
22300203,no,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,no,3,27497
22300204,no,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,no,3,27497


In [91]:
sec42 = pd.read_parquet('../../hces-2023-24/data/sec-4.2-CSQ.parquet')
sec42.head()

,procured_kerosene_ration,received_subsidy_lpg,num_subsidized_lpg,received_free_electricity,is_hhmem_edu,num_hhmem_publicedu,num_hhmem_privedu,received_free_textbook,num_free_textbook,received_free_stationary,...,is_hospitalization,is_benefit_healthscheme,num_benefit_healthscheme,amt_benefit_healthscheme,online_fuel,online_toilet,online_edu,online_medicine,online_services,multiplier
hhid,,,,,,,,,,,,,,,,,,,,,
22300101,2,1,2.0,2,2,NaN,NaN,NaN,NaN,NaN,...,4,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,57436
22300201,2,1,2.0,2,2,NaN,NaN,NaN,NaN,NaN,...,4,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,27497
22300202,2,1,1.0,2,2,NaN,NaN,NaN,NaN,NaN,...,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,27497
22300203,2,2,NaN,2,2,NaN,NaN,NaN,NaN,NaN,...,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27497
22300204,2,1,1.0,2,2,NaN,NaN,NaN,NaN,NaN,...,4,NaN,NaN,NaN,1.0,NaN,NaN,1.0,1.0,27497


In [92]:
sec43 = pd.read_parquet('../../hces-2023-24/data/sec-4.3-DGQ.parquet')
sec43.head()

,online_clothing,online_footwear,online_furniture,online_mobile,online_personal_goods,online_rec,online_cooking,online_crockery,online_sports,online_medical,...,has_bicycle,has_bike,has_car,has_truck,has_animalcart,has_fridge,has_washingmachine,has_ac,type_tv,multiplier
hhid,,,,,,,,,,,,,,,,,,,,,
22300101,1.0,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,1.0,NaN,NaN,1.0,1.0,1.0,2.0,57436
22300201,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,1.0,NaN,NaN,1.0,1.0,1.0,NaN,27497
22300202,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,1.0,NaN,NaN,1.0,1.0,1.0,2.0,27497
22300203,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,1.0,NaN,NaN,1.0,1.0,1.0,2.0,27497
22300204,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,1.0,NaN,NaN,1.0,1.0,1.0,2.0,27497


In [93]:
sec42.rename({'received_free_oth': 'received_free_oth_school', 'num_free_oth': 'num_free_oth_school'}, axis=1, inplace=True)
sec43.rename({'received_free_oth': 'received_free_oth_dgq', 'num_free_oth': 'num_free_oth_dgq'}, axis=1, inplace=True)

In [94]:
sec4 = pd.concat([sec41, sec42.drop(['multiplier'], axis=1), sec43.drop(['multiplier'], axis=1)], axis=1)
sec4.to_parquet('data/hh-procurement.parquet')

---

## Consumption and Expenditure

In [110]:
# Foods
sec51 = pd.read_parquet('../../hces-2023-24/data/sec-5.1-FDQ.parquet')
sec51.head(10)

cons_home_qty  cons_home_value  cons_total_qty  \
hhid     item_code                                                   
22300101 129                  NaN              NaN           27.60   
         112                  NaN              NaN            0.35   
         108                  NaN              NaN            2.00   
         102                  NaN              NaN           25.00   
         105                  NaN              NaN            0.25   
22300201 112                  NaN              NaN            0.50   
         129                  NaN              NaN           26.85   
         105                  NaN              NaN            0.35   
         102                  NaN              NaN           23.00   
         108                  NaN              NaN            3.00   

                    cons_total_value  source  multiplier  \
hhid     item_code                                         
22300101 129                  1687.0     NaN       57436   
         112                    30.0     1.0       57436   
         108                   132.0     1.0       57436   
         102                  1500.0     1.0       57436   
         105                    25.0     1.0       57436   
22300201 112                    50.0     1.0       27497   
         129                  1665.0     NaN       27497   
         105                    35.0     1.0       27497   
         102                  1380.0     1.0       27497   
         108                   200.0     1.0       27497   

                                   item_name  
hhid     item_code                            
22300101 129                cereal sub-total  
         112                vermicelli sewai  
         108        wheat atta other sources  
         102              rice other sources  
         105                            muri  
22300201 112                vermicelli sewai  
         129                cereal sub-total  
         105                            muri  
         102              rice other sources  
         108        wheat atta other sources

In [98]:
sec52 = pd.read_parquet('../../hces-2023-24/data/sec-5.2-FDQ.parquet')
sec52.head()

cons_home_qty  cons_home_value  cons_total_qty  \
hhid     item_code                                                   
22300101 152                  NaN              NaN            0.10   
         159                  NaN              NaN            3.85   
         143                  NaN              NaN            0.50   
         142                  NaN              NaN            1.00   
         145                  NaN              NaN            1.00   

                    cons_total_value  source  multiplier  \
hhid     item_code                                         
22300101 152                    12.0     1.0       57436   
         159                   562.0     NaN       57436   
         143                    75.0     1.0       57436   
         142                   100.0     1.0       57436   
         145                   160.0     1.0       57436   

                                                   item_name  
hhid     item_code                                            
22300101 152        other pulse products (soya chunks, etc.)  
         159             pulses & pulse products - sub-total  
         143                                           moong  
         142                                    gram (whole)  
         145                                             urd

In [99]:
sec53 = pd.read_parquet('../../hces-2023-24/data/sec-5.3-FDQ.parquet')
sec53.head()

cons_home_qty  cons_home_value  cons_total_qty  \
hhid     item_code                                                   
22300101 170                  NaN              NaN             1.0   
         179                  NaN              NaN             3.5   
         172                  NaN              NaN             2.5   
22300201 172                  NaN              NaN             2.5   
         170                  NaN              NaN             1.0   

                    cons_total_value  source  multiplier  \
hhid     item_code                                         
22300101 170                    25.0     1.0       57436   
         179                   125.0     NaN       57436   
         172                   100.0     1.0       57436   
22300201 172                   100.0     1.0       27497   
         170                    25.0     1.0       27497   

                                   item_name  
hhid     item_code                            
22300101 170            salt - other sources  
         179        salt & sugar - sub-total  
         172           sugar - other sources  
22300201 172           sugar - other sources  
         170            salt - other sources

In [112]:
sec5 = pd.concat((sec51, sec52, sec53), axis=0, verify_integrity=True).sort_index()

In [114]:
sec5.to_parquet('data/01-ce-cereals-pulses-sugar-salt.parquet')

In [117]:
# other food
files = !ls ../../hces-2023-24/data/sec-6.*.parquet
sec6 = []
for file in files:
    sec6.append(pd.read_parquet(file))
sec6 = pd.concat(sec6, axis=0, verify_integrity=True).sort_index()

In [118]:
sec6.to_parquet('data/02-ce-other-foods.parquet')

In [119]:
# processed food
files = !ls ../../hces-2023-24/data/sec-7.*.parquet
sec7 = []
for file in files:
    sec7.append(pd.read_parquet(file))
sec7 = pd.concat(sec7, axis=0, verify_integrity=True).sort_index()

In [120]:
sec7.to_parquet('data/03-ce-processed-food.parquet')

In [123]:
# energy
!cp ../../hces-2023-24/data/sec-8.1-CSQ.parquet ./data/04-ce-energy-utilities.parquet